### Assignment 21: KNearestNeighbor Regression

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

##### Load the Dataset

In [3]:
df = pd.read_csv('nba_2013.csv')

##### About the Dataset

In [4]:
df.shape

(481, 31)

In [5]:
col = df.columns
print(col)

Index(['player', 'pos', 'age', 'bref_team_id', 'g', 'gs', 'mp', 'fg', 'fga',
       'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft',
       'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'pts', 'season', 'season_end'],
      dtype='object')


In [6]:
# Features needed for Analysis

need_cols = ['pos', 'age', 'g', 'gs', 'mp', 'fg.', 'x3p.', 'x2p.', 'efg.', 'ft.', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']

In [7]:
data = df[need_cols]
data.head()

,pos,age,g,gs,mp,fg.,x3p.,x2p.,efg.,ft.,orb,drb,ast,stl,blk,tov,pf,pts
0,SF,23,63,0,847,0.468,0.266667,0.492063,0.482,0.660,72,144,28,23,26,30,122,171
1,C,20,81,20,1197,0.503,NaN,0.502703,0.503,0.581,142,190,43,40,57,71,203,265
2,PF,27,53,12,961,0.520,NaN,0.520000,0.520,0.639,102,204,38,24,36,39,108,362
3,SG,28,73,73,2552,0.459,0.426667,0.472574,0.522,0.815,32,230,248,35,3,146,136,1330
4,C,25,56,30,951,0.546,0.000000,0.548387,0.546,0.836,94,183,40,23,46,63,187,328


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pos     481 non-null    object 
 1   age     481 non-null    int64  
 2   g       481 non-null    int64  
 3   gs      481 non-null    int64  
 4   mp      481 non-null    int64  
 5   fg.     479 non-null    float64
 6   x3p.    414 non-null    float64
 7   x2p.    478 non-null    float64
 8   efg.    479 non-null    float64
 9   ft.     461 non-null    float64
 10  orb     481 non-null    int64  
 11  drb     481 non-null    int64  
 12  ast     481 non-null    int64  
 13  stl     481 non-null    int64  
 14  blk     481 non-null    int64  
 15  tov     481 non-null    int64  
 16  pf      481 non-null    int64  
 17  pts     481 non-null    int64  
dtypes: float64(5), int64(12), object(1)
memory usage: 67.8+ KB


In [9]:
data.describe()

,age,g,gs,mp,fg.,x3p.,x2p.,efg.,ft.,orb,drb,ast,stl,blk,tov,pf,pts
count,481.000000,481.000000,481.000000,481.000000,479.000000,414.000000,478.000000,479.000000,461.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000
mean,26.509356,53.253638,25.571726,1237.386694,0.436436,0.285111,0.466947,0.480752,0.722419,55.810811,162.817048,112.536383,39.280665,24.103950,71.862786,105.869023,516.582121
std,4.198265,25.322711,29.658465,897.258840,0.098672,0.157633,0.104448,0.099552,0.160166,62.101191,145.348116,131.019557,34.783590,30.875381,62.701690,71.213627,470.422228
min,19.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,32.000000,0.000000,388.000000,0.400500,0.234355,0.434719,0.451000,0.654000,12.000000,43.000000,20.000000,9.000000,4.000000,21.000000,44.000000,115.000000
50%,26.000000,61.000000,10.000000,1141.000000,0.438000,0.330976,0.474475,0.488000,0.751000,35.000000,135.000000,65.000000,32.000000,14.000000,58.000000,104.000000,401.000000
75%,29.000000,76.000000,54.000000,2016.000000,0.479500,0.375000,0.513729,0.526000,0.821000,73.000000,230.000000,152.000000,60.000000,32.000000,108.000000,158.000000,821.000000
max,39.000000,83.000000,82.000000,3122.000000,1.000000,1.000000,1.000000,1.000000,1.000000,440.000000,783.000000,721.000000,191.000000,219.000000,295.000000,273.000000,2593.000000


##### Checking for Missing values

In [10]:
data.isnull().sum()

pos      0
age      0
g        0
gs       0
mp       0
fg.      2
x3p.    67
x2p.     3
efg.     2
ft.     20
orb      0
drb      0
ast      0
stl      0
blk      0
tov      0
pf       0
pts      0
dtype: int64

In [11]:
data.fillna(0,inplace=True)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


##### Standardizing Numeric Features

In [12]:
num_cols = need_cols
num_cols.remove('pos')

In [13]:
scaler = StandardScaler()
scaler.fit(data[num_cols])
scale_data = scaler.transform(data[num_cols])

In [14]:
scale_data = pd.DataFrame(scale_data, columns = num_cols)
scale_data.head()

,age,g,gs,mp,fg.,x3p.,x2p.,efg.,ft.,orb,drb,ast,stl,blk,tov,pf,pts
0,-0.836776,0.385287,-0.863104,-0.435541,0.326296,0.120645,0.254072,0.031240,-0.152085,0.260962,-0.129597,-0.645891,-0.468543,0.061474,-0.668345,0.226751,-0.735386
1,-1.552101,1.096851,-0.188059,-0.045058,0.668445,-1.391945,0.350514,0.233258,-0.523132,1.389328,0.187214,-0.531285,0.020702,1.066555,-0.013774,1.365358,-0.535357
2,0.116990,-0.010027,-0.458077,-0.308355,0.834631,-1.391945,0.507309,0.396797,-0.250717,0.744547,0.283635,-0.569487,-0.439764,0.385694,-0.524659,0.029955,-0.328945
3,0.355432,0.780600,1.600813,1.466669,0.238315,1.028199,0.077404,0.416036,0.575919,-0.383819,0.462702,1.034996,-0.123194,-0.684232,1.183611,0.423547,1.730923
4,-0.359893,0.108567,0.149464,-0.319512,1.088799,-1.391945,0.764630,0.646915,0.674552,0.615591,0.139004,-0.554207,-0.468543,0.709913,-0.141496,1.140448,-0.401296


##### Encoding Categorical Feature

In [15]:
data_dummies = pd.get_dummies(data['pos'], columns=['pos'],drop_first=True)
data_dummies.head()

,F,G,PF,PG,SF,SG
0,0,0,0,0,1,0
1,0,0,0,0,0,0
2,0,0,1,0,0,0
3,0,0,0,0,0,1
4,0,0,0,0,0,0


##### Merging Standardized and Encoded Features

In [16]:
data = scale_data.merge(data_dummies, left_index=True, right_index=True)

In [17]:
data.shape

(481, 23)

In [18]:
data.head()

,age,g,gs,mp,fg.,x3p.,x2p.,efg.,ft.,orb,...,blk,tov,pf,pts,F,G,PF,PG,SF,SG
0,-0.836776,0.385287,-0.863104,-0.435541,0.326296,0.120645,0.254072,0.031240,-0.152085,0.260962,...,0.061474,-0.668345,0.226751,-0.735386,0,0,0,0,1,0
1,-1.552101,1.096851,-0.188059,-0.045058,0.668445,-1.391945,0.350514,0.233258,-0.523132,1.389328,...,1.066555,-0.013774,1.365358,-0.535357,0,0,0,0,0,0
2,0.116990,-0.010027,-0.458077,-0.308355,0.834631,-1.391945,0.507309,0.396797,-0.250717,0.744547,...,0.385694,-0.524659,0.029955,-0.328945,0,0,1,0,0,0
3,0.355432,0.780600,1.600813,1.466669,0.238315,1.028199,0.077404,0.416036,0.575919,-0.383819,...,-0.684232,1.183611,0.423547,1.730923,0,0,0,0,0,1
4,-0.359893,0.108567,0.149464,-0.319512,1.088799,-1.391945,0.764630,0.646915,0.674552,0.615591,...,0.709913,-0.141496,1.140448,-0.401296,0,0,0,0,0,0


##### Model Building: KNN Regressor

In [19]:
X = data.loc[:, data.columns!='pts']
y = data.loc[:,'pts']

In [20]:
# Splitting dataset into train and test dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.20)

In [21]:
model = KNeighborsRegressor(n_neighbors = 5)
model.fit(X_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [22]:
# Predicting for Test dataset

pred = model.predict(X_test)

In [23]:
# Score of the model: Coefficient of Determination

model.score(X_test, y_test)

0.8829094156951394